In [1]:
%load_ext autoreload
%autoreload 2
%autosave 60
%matplotlib inline
from ivanocode.ivanocommon import *
import pandas as pd

Autosaving every 60 seconds


In [69]:
raw = 'raw'
submissions = 'submissions'

In [3]:
df_sample_submission = pd.read_csv(f'{raw}/sample_submission.csv')
df_sample_submission.columns

Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object')

In [4]:
df_sales_train_melt = pd.read_feather('df_sales_train_melt.feather')

In [5]:
df_sales_train_melt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,daily_avg_count,monthly_avg_count,day_date,day_date_str,month_id,month,year,date,wm_yr_wk,sell_price,sell_price_flip_count
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0.313643,9.409305,2011-01-29,2011-01-29,1,1,2011,2011-01-29,11101,NaN,1


In [6]:
days = df_sales_train_melt['day_id'].unique()
days[0], days[-1]

(1, 1913)

In [7]:
# F1 = d_1914, ... F28 = d_1941

1941

In [66]:
query = ("id == 'HOBBIES_1_001_CA_1_validation' or "+
         "id == 'HOBBIES_1_002_CA_1_validation' or "+
         "id == 'HOBBIES_1_004_CA_1_validation'")
subset = df_sales_train_melt.query(query).copy()

In [76]:
%%time
min_day = 1885
df_submission = (df_sales_train_melt[['id', 'sales', 'day_id']]
    .groupby('id', as_index=False)
    .tail(28)
    .pivot(index='id', columns='day_id', values='sales')
    .rename(lambda x: f'F{x-min_day}', axis=1)
    .reset_index()
    .sort_values('id')
)

CPU times: user 5.81 s, sys: 2.16 s, total: 7.98 s
Wall time: 8.03 s


In [87]:
eval_rows = df_sample_submission[df_sample_submission['id'].str.contains('_evaluation')]

In [88]:
pd.concat([df_submission, eval_rows]).to_csv(f'{submissions}/0200-naive.csv', index=False)

In [89]:
!head -n2 {submissions}/0200-naive.csv

id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
FOODS_1_001_CA_1_validation,2,1,1,0,4,0,0,4,1,3,0,1,0,2,2,0,1,1,0,2,0,4,1,1,0,1,1,0


In [90]:
!head -n2 {raw}/sample_submission.csv

id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [91]:
!wc -l {submissions}/0200-naive.csv

   60981 submissions/0200-naive.csv


In [82]:
!wc -l {raw}/sample_submission.csv

   60981 raw/sample_submission.csv


In [83]:
!open {submissions}

In [84]:
!open https://www.kaggle.com/c/m5-forecasting-accuracy/submit